## CIFAR

In [1]:
import torch
import numpy as np
from tqdm import tqdm
import os
from PIL import Image
from transformers import ViTForImageClassification, ViTImageProcessor
import cv2
import pickle

2024-12-20 19:37:41.673982: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0
/usr/local/lib/python3.8/dist-packages/tensorflow/python/framework/dtypes.py:513: FutureWarning: In the future `np.object` will be defined as the corresponding NumPy scalar.
  np.object,


RuntimeError: Failed to import transformers.models.vit.image_processing_vit because of the following error (look up to see its traceback):
module 'numpy' has no attribute 'object'.
`np.object` was a deprecated alias for the builtin `object`. To avoid this error in existing code, use `object` by itself. Doing this will not modify any behavior and is safe. 
The aliases was originally deprecated in NumPy 1.20; for more details and guidance see the original release note at:
    https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations

In [ ]:
from utils import cifar100_classes, crop_with_context, get_img

# Classify each object using VIT-cifar

In [ ]:
extractor = ViTImageProcessor.from_pretrained('google/vit-base-patch16-224-in21k')
model = ViTForImageClassification.from_pretrained('edumunozsala/vit_base-224-in21k-ft-cifar100')

# Open annotations
with open('../../resources/annotations_public.pkl', 'rb') as f:
    anns = pickle.load(f)

def process(image, objects):
    for obj in objects:
        img_cropped = crop_with_context(image, obj['bbox'], 0.2)
        inputs = extractor(images=img_cropped, return_tensors="pt").to('cuda')

        with torch.no_grad():
            outputs = model(**inputs)
        
        probabilities = torch.nn.functional.softmax(outputs.logits.cpu(), dim=-1)
        prob, idx = probabilities.topk(10)
        top10_class = cifar100_classes[idx[0]]
        top10_probs = prob[0].numpy()

        obj['top10_probs'] = top10_class
        obj['top10_class'] = top10_probs
    return objects


folder_path = "../data"

results = {}
for filename in tqdm(os.listdir(folder_path)):
    if filename.endswith(".mp4"):
        video = filename.split('.')[0]
        cap = cv2.VideoCapture(os.path.join(folder_path, filename))
        original_fps = cap.get(cv2.CAP_PROP_FPS)

        # Initialize storage for this video
        video_results = {}

        frame_count = 0
        while cap.isOpened():
            ret, frame = cap.read()
            if not ret:
                break

            # Process every Nth frame to match the target FPS
            image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            objects = anns[video][frame_count]['challenge_object']
            video_results[frame_count] = process(image, objects)
            frame_count += 1

        cap.release()
        results[video] = video_results

torch.save(results, f'../../resources/cifar-classification/all-dense.pkl')

100%|██████████| 201/201 [34:52<00:00, 10.41s/it] 
